In [1]:
from pyabsa import ABSADatasetList, available_checkpoints
from pyabsa import ATEPCCheckpointManager

Version 1.7.9.7 of findfile is outdated. Version 1.7.9.8 was released Friday July 22, 2022.
This script could only be used to manage NVIDIA GPUs,but no GPU found in your device


2022-08-16 17:58:59.035223: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-16 17:58:59.035258: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/keras/utils/image_utils.py:36: DeprecationWarning: NEAREST is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  'nearest': pil_image.NEAREST,
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/keras/utils/image_utils.py:37: DeprecationWarning: BILINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BILINEAR instead.
  'bilinear': pil_image.BILINEAR,
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/keras/utils/ima

Version 0.4.32 of metric_visualizer is outdated. Version 0.5.3 was released 1 day ago.
Remote ABSADataset version: 2022.07.22 Local ABSADatasets version: 2022.07.22
Version 1.16.7 of pyabsa is outdated. Version 1.16.9 was released 8 hours ago.
check release notes at https://github.com/yangheng95/PyABSA/blob/release/release-note.json


In [2]:
# Use pretrained model 
aspect_extractor = ATEPCCheckpointManager.get_aspect_extractor(checkpoint='english', auto_device=False) # False means load model on CPU

Load aspect extractor from checkpoints/ATEPC_ENGLISH_CHECKPOINT
config: checkpoints/ATEPC_ENGLISH_CHECKPOINT/fast_lcf_atepc.config
state_dict: checkpoints/ATEPC_ENGLISH_CHECKPOINT/fast_lcf_atepc.state_dict
model: None
tokenizer: checkpoints/ATEPC_ENGLISH_CHECKPOINT/fast_lcf_atepc.tokenizer


Some weights of the model checkpoint at microsoft/deberta-v3-base were not used when initializing DebertaV2Model: ['lm_predictions.lm_head.dense.weight', 'mask_predictions.LayerNorm.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'mask_predictions.LayerNorm.weight', 'lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.dense.weight', 'lm_predictions.lm_head.bias', 'lm_predictions.lm_head.dense.bias', 'mask_predictions.classifier.weight', 'mask_predictions.classifier.bias', 'mask_predictions.dense.bias']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Special tokens have b

Config used in Training:
ABSADatasetsVersion:None	-->	Calling Count:0
IOB_label_to_index:{'B-ASP': 1, 'I-ASP': 2, 'O': 3, '[CLS]': 4, '[SEP]': 5}	-->	Calling Count:1
MV:<metric_visualizer.metric_visualizer.MetricVisualizer object at 0x7f78046c4910>	-->	Calling Count:0
PyABSAVersion:1.15.7	-->	Calling Count:0
SRD:3	-->	Calling Count:79070
TorchVersion:1.10.0+cuda11.3	-->	Calling Count:0
TransformersVersion:4.20.1	-->	Calling Count:0
auto_device:True	-->	Calling Count:50569
batch_size:16	-->	Calling Count:5
cache_dataset:True	-->	Calling Count:1
dataset_file:{'train': ['integrated_datasets/atepc_datasets/110.SemEval/113.laptop14/Laptops_Train.xml.seg.atepc', 'integrated_datasets/atepc_datasets/110.SemEval/114.restaurant14/Restaurants_Train.xml.seg.atepc', 'integrated_datasets/atepc_datasets/110.SemEval/116.restaurant16/restaurant_train.raw.atepc', 'integrated_datasets/atepc_datasets/101.ACL_Twitter/acl-14-short-data/train.raw.atepc', 'integrated_datasets/atepc_datasets/109.MAMS/train.xml

---

In [3]:
import pandas as pd

# Load data
restaurants_reviews = pd.read_parquet('restaurants_reviews')
users_reviews = pd.read_parquet('users_reviews')

In [4]:
restaurants_reviews.reset_index(drop=True, inplace=True)
users_reviews.reset_index(drop=True, inplace=True)

In [5]:
restaurants_reviews_text = restaurants_reviews['text']
users_reviews_text = users_reviews['text']

In [1]:
from nltk.stem.wordnet import WordNetLemmatizer
lemma = WordNetLemmatizer()

In [7]:
def preprocess(review):
    sentences = [sentence.strip().lower() for sentence in review.replace('\n', '').split('.')] # remove next-line and split by '.'
    sentences_handled = []
    for i, sentence in enumerate(sentences):
        tokens = [lemma.lemmatize(lemma.lemmatize(token, pos='v'), pos='n') for token in sentence.split(' ') if token]
        if len(tokens) > 0:
            sentences_handled.append(' '.join(tokens))
    return sentences_handled

In [8]:
restaurants_reviews_sentences = restaurants_reviews_text.apply(preprocess)
users_reviews_sentences = users_reviews_text.apply(preprocess)

In [9]:
from IPython.display import clear_output as clear
count = 0
length = restaurants_reviews_sentences.shape[0]

def innerFunc(sentences):
    global count
    if count % 5 == 0:
        clear()
        print('Process:', f'{count}/{length}')
    count += 1
    result = aspect_extractor.extract_aspect(inference_source=sentences, save_result=False, print_result=False, pred_sentiment=True)
    return list(map(lambda item: {'aspect': item['aspect'], 'sentiment': item['sentiment']}, result))

In [ ]:
restaurants_aspects_with_polarities = restaurants_reviews_sentences.apply(innerFunc)

Process: 4120/20000


In [ ]:
restaurants_reviews['aspect_polarity'] = restaurants_aspects_with_polarities

In [ ]:
restaurants_reviews.to_parquet('restaurants_reviews_with_aspects_polarities')

In [ ]:
count = 0
length = users_reviews_sentences.shape[0]
users_aspects_with_polarities = users_reviews_sentences.apply(innerFunc)

In [ ]:
users_reviews['aspect_polarity'] = users_aspects_with_polarities

In [ ]:
users_reviews.to_parquet('users_reviews_with_aspects_polarities')